# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results, and you'll learn methods to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Analyze the results of regression and R-squared and adjusted R-squared 
* Understand and apply forward and backward predictor selection

## The Boston Housing Data once more

We pre-processed the Boston Housing data the same way we did before:

- We dropped `'ZN'` and `'NOX'` completely
- We categorized `'RAD'` in 3 bins and `'TAX'` in 4 bins
- We transformed `'RAD'` and `'TAX'` to dummy variables and dropped the first variable
- We used min-max-scaling on `'B'`, `'CRIM'`, and `'DIS'` (and logtransformed all of them first, except `'B'`)
- We used standardization on `'AGE'`, `'INDUS'`, `'LSTAT'`, and `'PTRATIO'` (and logtransformed all of them first, except for `'AGE'`) 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(['NOX', 'ZN'],axis=1)

# First, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# First, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix='TAX', drop_first=True)
rad_dummy = pd.get_dummies(bins_rad, prefix='RAD', drop_first=True)
boston_features = boston_features.drop(['RAD', 'TAX'], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features['AGE']
b = boston_features['B']
logcrim = np.log(boston_features['CRIM'])
logdis = np.log(boston_features['DIS'])
logindus = np.log(boston_features['INDUS'])
loglstat = np.log(boston_features['LSTAT'])
logptratio = np.log(boston_features['PTRATIO'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['CRIM'] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['AGE'] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features['INDUS'] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features['PTRATIO'] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

## Perform stepwise selection

The function for stepwise selection is copied below. Use this function provided on your preprocessed Boston Housing data.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [4]:
# Your code here
y = boston.target
X=boston_features

In [5]:
stepwise_selection(X,y)

Add  LSTAT                          with p-value 9.27989e-122
Add  RM                             with p-value 1.98621e-16
Add  PTRATIO                        with p-value 2.5977e-12
Add  DIS                            with p-value 2.85496e-09
Add  B                              with p-value 2.77572e-06
Add  INDUS                          with p-value 0.0017767
Add  CHAS                           with p-value 0.0004737


['LSTAT', 'RM', 'PTRATIO', 'DIS', 'B', 'INDUS', 'CHAS']

### Build the final model again in Statsmodels

In [7]:
# Your code here
X = X.loc[:,['LSTAT', 'RM', 'PTRATIO', 'DIS', 'B', 'INDUS', 'CHAS']]

In [11]:
model = sm.OLS(y,X).fit()

In [12]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     2121.
Date:                Sun, 13 Oct 2019   Prob (F-statistic):               0.00
Time:                        22:14:14   Log-Likelihood:                -1466.3
No. Observations:                 506   AIC:                             2947.
Df Residuals:                     499   BIC:                             2976.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
LSTAT         -5.3059      0.257    -20.607      0.000      -5.812      -4.800
RM             3.5154      0.135     26.012      0.000       3.250       3.781
PTRATIO       -1.3364      0.227     -5.889      0.000      -1.782      -0.891
DIS           -8.9280      1.250     -7.140      0.000     -11.385      -6.471
B              4.7139      0.861      5.476      0.000       3.023       6.405
INDUS         -1.0499      0.323     -3.255      0.001      -1.684      -0.416
CHAS           2.7523      0.797      3.455      0.001       1.187       4.318
==============================================================================
Omnibus:                      116.755   Durbin-Watson:                   1.082
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              545.114
Skew:                           0.934   Prob(JB):                    4.26e-119
Kurtosis:                       7.729   Cond. No.                         42.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The stepwise procedure mentions that `'INDUS'` was added with a p-value of 0.0017767, but our statsmodels output returns a p-value of 0.000. Use some of the stepwise procedure logic to find the intuition behind this!

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [13]:
# Your code here
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression


In [14]:
linreg = LinearRegression()
selector = RFE(linreg,n_features_to_select=5)

In [15]:
selector = selector.fit(X,y)

In [16]:
selector.support_

array([ True,  True, False,  True,  True, False,  True])

In [18]:
selector.ranking_

array([1, 1, 2, 1, 1, 3, 1])

In [20]:
estimators = selector.estimator

In [25]:
estimators = selector.estimator_
estimators.coef_
#estimators.intercept_


array([-6.25217488,  3.43718997, -6.58036332,  4.65357304,  2.93498961])

In [27]:
X.head()

,LSTAT,RM,PTRATIO,DIS,B,INDUS,CHAS
0,-1.275260,6.575,-1.443977,0.542096,1.000000,-1.704344,0.0
1,-0.263711,6.421,-0.230278,0.623954,1.000000,-0.263239,0.0
2,-1.627858,7.185,-0.230278,0.623954,0.989737,-0.263239,0.0
3,-2.153192,6.998,0.165279,0.707895,0.994276,-1.778965,0.0
4,-1.162114,7.147,0.165279,0.707895,1.000000,-1.778965,0.0


Fit the linear regression model again using the 5 selected columns

In [28]:
# Your code here
X = X.loc[:,['LSTAT', 'RM', 'DIS', 'B', 'CHAS']]

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [29]:
# Your code here
linreg = LinearRegression()

In [30]:
model = linreg.fit(X,y)

In [35]:
model.coef_,model.intercept_

(array([-6.25217488,  3.43718997, -6.58036332,  4.65357304,  2.93498961]),
 -0.4973982153788974)

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [ ]:
# Your code here

# r_squared is 0.742981  
# adjusted_r_squared is 0.740411

## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You now performed your own feature selection methods!